In [1]:
import pandas as pd
import numpy as np
from langchain.agents import initialize_agent, Tool
from langchain.llms.openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import make_classification
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_experimental.tools.python.tool import PythonREPLTool
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
@tool
def handle_missing_values(df_str):
    """Handle missing values in the dataset based on df type."""
    data = globals()[df_str.split('\n')[0]]
    # print("The df type that enters the functions: " + str(type(df)))
    # data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/lanchain_1/titanic.csv')
    # df = pd.from_dict(data)
    # print(df)
    # df = convert_dict_to_DataFrame(df)
    # print("The df that enters the functions: " + df)
    #print(df.head)
    try:

        # Separate numeric and categorical columns
        numeric_cols = data.select_dtypes(include=['number']).columns
        categorical_cols = data.select_dtypes(include=['object', 'category']).columns

        # Handle missing values for numeric columns
        if numeric_cols.size > 0:
            numeric_imputer = SimpleImputer(strategy='mean')
            data[numeric_cols] = numeric_imputer.fit_transform(data[numeric_cols])

        # Handle missing values for categorical columns
        if categorical_cols.size > 0:
            categorical_imputer = SimpleImputer(strategy='most_frequent')
            data[categorical_cols] = categorical_imputer.fit_transform(data[categorical_cols])

    except Exception as e:
        print(f"Error in handle_missing_values: {e}")
    globals()[df_str.split('\n')[0]] = data
    data.to_csv('transformed_df.csv', index=False)
    return data
    # return "Handled missing values in numerical columns: " + str(numeric_cols) + " and in categorical columns: " + str(categorical_cols)

@tool
def encode_categorical_features(df_str):
    """Encode categorical features with one-hot encoding or label encoding based on the number of unique categories."""
    data = globals()[df_str.split('\n')[0]]
    max_categories = 10
    all_encoded_cols = ""
    all_labeled_cols = ""
    try:
        # Ensure the DataFrame contains categorical columns
        categorical_cols = data.select_dtypes(include=['object', 'category']).columns

        if categorical_cols.size > 0:
            for col in categorical_cols:
                num_unique = data[col].nunique()

                # Apply Label Encoding if the number of unique categories is too high
                if num_unique > max_categories:
                    print(f"Applying Label Encoding to '{col}' with {num_unique} unique categories.")
                    label_encoder = LabelEncoder()
                    data[col] = label_encoder.fit_transform(data[col])
                    all_labeled_cols = all_labeled_cols + " " + col

                else:
                    print(f"Applying One-Hot Encoding to '{col}' with {num_unique} unique categories.")
                    # Apply One-Hot Encoding for columns with a manageable number of categories
                    encoder = OneHotEncoder(sparse_output=False, drop='first')
                    encoded_array = encoder.fit_transform(data[[col]])
                    encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out([col]))
                    data = data.drop(columns=[col])
                    data = pd.concat([data, encoded_df], axis=1)
                    all_encoded_cols = all_encoded_cols + " " + col
        else:
            print("No categorical columns to encode.")

    except Exception as e:
        print(f"Error in encode_categorical_features: {e}")

    globals()[df_str.split('\n')[0]] = data
    data.to_csv('transformed_df.csv', index=False)
    return data
    # return "done encoding"


@tool
def scale_features(df_str):
    """Scale numerical features using StandardScaler."""
    # df = convert_dict_to_DataFrame(df)
    data = globals()[df_str.split('\n')[0]]
    try:
        # Ensure the DataFrame contains numerical df
        numeric_cols = data.select_dtypes(include=['number']).columns
        if numeric_cols.size > 0:
            scaler = StandardScaler()
            data[numeric_cols] = scaler.fit_transform(data[numeric_cols])
        else:
            print("No numerical columns to scale.")
    except Exception as e:
        print(f"Error in scale_features: {e}")
    # data.to_csv('transformed_df.csv', index=False)
    globals()[df_str.split('\n')[0]] = data
    data.to_csv('transformed_df.csv', index=False)
    return data
    # return "Done scaling numerical features."

In [3]:
df = pd.read_csv('titanic.csv')
df_types = str(df.dtypes).split('\n')
df_types = [i.split('     ') for i in df_types]
df_types

[['PassengerId', ' int64'],
 ['Survived', '    int64'],
 ['Pclass', '', ' int64'],
 ['Name', '', '  object'],
 ['Sex', '', '   object'],
 ['Age', '', '  float64'],
 ['SibSp', '', '  int64'],
 ['Parch', '', '  int64'],
 ['Ticket', '', 'object'],
 ['Fare', '', ' float64'],
 ['Cabin', '', ' object'],
 ['Embarked', '   object'],
 ['dtype: object']]

In [4]:
# Initialize an empty dictionary
column_types = {}

# Process each row in the 2D array
for row in df_types:
    # The column name is the first element
    column_name = row[0].strip()

    # The data type is the last non-empty element
    data_type = None
    for item in reversed(row[1:]):
        if item.strip():
            data_type = item.strip()
            break

    # Only add to dictionary if we have both column name and data type
    if column_name and data_type:
        column_types[column_name] = data_type

# Remove any unwanted entries like 'dtype: object'
column_types.pop('dtype: object', None)

# Print the result
print(column_types)
column_types_string = str(column_types)

{'PassengerId': 'int64', 'Survived': 'int64', 'Pclass': 'int64', 'Name': 'object', 'Sex': 'object', 'Age': 'float64', 'SibSp': 'int64', 'Parch': 'int64', 'Ticket': 'object', 'Fare': 'float64', 'Cabin': 'object', 'Embarked': 'object'}


In [5]:
tools = [
    Tool(
        name="handle_missing_values",
        func=handle_missing_values,
        description="This is a tool that can help handling missing values in a dataset. It can only take input 'df'. It imputes the numerical columns with mean values and the categorical values with most frequent",
    ),
    Tool(
        name="scale_features",
        func=scale_features,
        description="Scale numerical features using StandardScaler to standardize features to have zero mean and unit variance. It can only take input 'df'.",
    ),
    Tool(
        name="encode_categorical_features",
        func=encode_categorical_features,
        description="One-hot encode categorical features, converting categorical values into binary vectors. It can only take input 'df'.",
    )
]

In [6]:
from langchain_experimental.tools.python.tool import PythonAstREPLTool

tools = [PythonAstREPLTool(globals={"df": df})] + tools

In [7]:
OPENAI_API_KEY="sk-proj-F8FCluwmCKifAkJVLC8DT3BlbkFJqjYO0WkIiry07dlMr0aa"

In [8]:
# Initialize the OpenAI model
llm = ChatOpenAI(model="gpt-4o", openai_api_key=OPENAI_API_KEY, temperature=0)

/Users/alymohamed/Desktop/langchain/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [9]:
# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True
)

/Users/alymohamed/Desktop/langchain/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [34]:
prompt = f"""
Task Description

Objective: Perform pre-processing techniques on a dataset. The goal is to identify and apply the best pre-processing techniques for each column in the dataset.
Steps to Follow:

1. The dataset:
   - You can find the dataset in the globals variables with name df.
   - Do not create a new dataset and make use of the one I provided with variable name: df
   - First ensure you can access the dataframe df.

2. dataset Structure:
   - {column_types_string}
   - You'll see here there is a string with dictionary structure that states the datatype of each and every column.
   - For example: 'PassengerId': 'int64' This means the column passengerId is of type int64.
   

3. Tools:
   - Make sure you make use of PythonAstREPLTool() I provided for all coding related purposes.
   - Now that you have the dataset ensure that you only call the functions if you want to apply to each and every one of them.
   - If needed make use of each and every tool function I provided for pre-processing purposes.
   - The tools are handle missing values, scaling features and encoding categorical features. 
   - These are the only tools you can use and you have access to: [python_repl_ast, handle_missing_values, scale_features, encode_categorical_features] 
   - In handle missing values the tool imputes numerical missing values with mean and imputes categorical columns with most_frequent. 
   - In encode_cateogrical_features tool label encoding is applied to columns with more than 10 categories and hot encoding is applied to columns with less than 10 categories. 

4. Implementation:
   - Use the PythonAstREPLTool to perform pre-processing techniques on the dataset.
   - Implement the transformations and verify their correctness using the tools, ensuring every function used is compatible with the specific df types of each column.
   - After each and every execution of the tool, ensure that the transformed dataset is saved back to the original variable df.

5. Output Requirements:
   - Save the transformed dataset back to csv after applying the relevant pre-processing techniques.
   - Ensure the transformed dataset is saved with the name 'transformed_dataset.csv'.
   - Make sure the changes are applied to the global variable df after using each and every tool.
   - Make sure the saved dataset is the final transformed dataset after applying all the pre-processing techniques.
"""


# Run the agent with the formatted input df
result = agent.run(prompt)
# Print the result (feature engineering strategies suggested)
print(result)



> Entering new AgentExecutor chain...
To perform the required pre-processing on the dataset, I will follow these steps:

1. Verify that the dataset `df` is accessible.
2. Handle missing values in the dataset.
3. Scale the numerical features.
4. Encode the categorical features.
5. Save the transformed dataset to a CSV file named 'transformed_dataset.csv'.

Let's start by verifying that the dataset `df` is accessible.

Action: python_repl_ast
Action Input: df.head()
Observation:    PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0  

In [10]:
transformed_data = pd.read_csv('transformed_df.csv')
transformed_data.shape

(891, 13)

In [11]:
prefix_prompt = """
You're an agent that will act as Data Scientist. You're assigned to do feature engineering techniques \n
"""

In [36]:
from langchain_experimental.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent
import pandas as pd
from langchain.agents import AgentType
from langchain_openai import ChatOpenAI
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.agents.agent_toolkits.pandas.base import _get_prompt
from langchain_experimental.tools.python.tool import PythonREPLTool
agent_pandas = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-4o", openai_api_key = OPENAI_API_KEY),
    df,
    prefix=prefix_prompt,
    verbose=True,
    allow_dangerous_code=True,
    agent_type="openai-tools",
    handle_parsing_errors=True
)

/Users/alymohamed/Desktop/langchain/.venv/lib/python3.11/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [30]:
prompt_template = _get_prompt(df)
prompt_template.template.split('\n')

['',
 'You are working with a pandas dataframe in Python. The name of the dataframe is `df`.',
 'You should use the tools below to answer the question posed of you:',
 '',
 '{tools}',
 '',
 'Use the following format:',
 '',
 'Question: the input question you must answer',
 'Thought: you should always think about what to do',
 'Action: the action to take, should be one of [{tool_names}]',
 'Action Input: the input to the action',
 'Observation: the result of the action',
 '... (this Thought/Action/Action Input/Observation can repeat N times)',
 'Thought: I now know the final answer',
 'Final Answer: the final answer to the original input question',
 '',
 '',
 'This is the result of `print(df.head())`:',
 '{df_head}',
 '',
 'Begin!',
 'Question: {input}',
 '{agent_scratchpad}']

In [37]:
prompt = """
Task: Feature Selection and Correlation Analysis \n
Select-K Best Features: Perform feature selection using the entire dataset (not just the first 5 rows) to identify the top K best features. Ensure you use the global variable df, which contains the dataset, for all operations.\n
\n
Build Correlation Matrices: Generate correlation matrices for the dataset to understand the relationships between different features.\n
\n
Interpret Results and Report Generation:\n
\n
Analyze the results to identify potential columns that can be used as target variables for a machine learning model.\n
For each potential target column, specify the most relevant features that could be used to predict that column based on the feature selection and correlation analysis.\n
Write your findings in a Markdown (.md) format, highlighting:\n
Possible target columns for prediction.\n
The most relevant features for predicting each target column.\n
Use the entire dataset (df) for all analyses and ensure your interpretation is clear and well-documented.\n
"""
result = agent_pandas.run(prompt)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "from sklearn.feature_selection import SelectKBest, f_classif\nimport pandas as pd\nimport numpy as np\n\n# Load the dataset\ndf = pd.read_csv('titanic.csv')\n\n# Drop rows with missing target values\ndf = df.dropna(subset=['Survived'])\n\n# Convert categorical variables to numeric\ncategorical_cols = ['Sex', 'Embarked']\ndf = pd.get_dummies(df, columns=categorical_cols, drop_first=True)\n\n# Fill missing values\nfor col in df.columns:\n    if df[col].isnull().sum() > 0:\n        if df[col].dtype == 'object':\n            df[col] = df[col].fillna(df[col].mode()[0])\n        else:\n            df[col] = df[col].fillna(df[col].median())\n\n# Define features and target\nX = df.drop(columns=['Survived', 'Name', 'Ticket', 'Cabin'])\ny = df['Survived']\n\n# Select K best features\nk = 5\nselector = SelectKBest(score_func=f_classif, k=k)\nselector.fit(X, y)\nselected_features = X.columns[selector.get_support(

Matplotlib is building the font cache; this may take a moment.


ValueError: could not convert string to float: 'Braund, Mr. Owen Harris'### Feature Selection and Correlation Analysis Report

#### Possible Target Columns for Prediction
Based on the dataset, the most obvious target column for prediction is `Survived`, which indicates whether a passenger survived the Titanic disaster.

#### Selected Features for Predicting `Survived`
Using the SelectKBest method with ANOVA F-value as the score function, the top 5 features selected for predicting the `Survived` column are:
1. **Pclass**: Passenger class (1st, 2nd, or 3rd)
2. **Parch**: Number of parents/children aboard the Titanic
3. **Fare**: Ticket fare
4. **Sex_male**: Gender of the passenger (male)
5. **Embarked_S**: Port of Embarkation (Southampton)

#### Correlation Matrix Analysis
The correlation matrix provides insights into the relationships between different features in the dataset. Below is the correlation matrix heatmap:

![Correlation Matrix](

> Finished chain.


'### Feature Selection and Correlation Analysis Report\n\n#### Possible Target Columns for Prediction\nBased on the dataset, the most obvious target column for prediction is `Survived`, which indicates whether a passenger survived the Titanic disaster.\n\n#### Selected Features for Predicting `Survived`\nUsing the SelectKBest method with ANOVA F-value as the score function, the top 5 features selected for predicting the `Survived` column are:\n1. **Pclass**: Passenger class (1st, 2nd, or 3rd)\n2. **Parch**: Number of parents/children aboard the Titanic\n3. **Fare**: Ticket fare\n4. **Sex_male**: Gender of the passenger (male)\n5. **Embarked_S**: Port of Embarkation (Southampton)\n\n#### Correlation Matrix Analysis\nThe correlation matrix provides insights into the relationships between different features in the dataset. Below is the correlation matrix heatmap:\n\n![Correlation Matrix]('

In [47]:
agent_output = """

### Feature Selection and Correlation Analysis Report

#### Possible Target Columns for Prediction
Based on the dataset, the most likely target column for prediction is `Survived`, which indicates whether a passenger survived or not.

#### Selected Features for Predicting `Survived`
Using the SelectKBest method, the top 5 features selected for predicting the `Survived` column are:
1. `Pclass`
2. `Ticket`
3. `Fare`
4. `Cabin`
5. `Sex_male`

#### Correlation Matrix Analysis
The correlation matrix provides insights into the relationships between different features. Here are some key observations:

- `Survived` has a strong negative correlation with `Sex_male` (-0.543351), indicating that males were less likely to survive.
- `Survived` has a moderate positive correlation with `Fare` (0.257307) and `Cabin` (0.175690), suggesting that passengers who paid higher fares and had cabins were more likely to survive.
- `Pclass` is negatively correlated with `Survived` (-0.338481), indicating that passengers in higher classes had a better chance of survival.

#### Most Relevant Features for Predicting `Survived`
Based on the feature selection and correlation analysis, the most relevant features for predicting the `Survived` column are:
1. `Sex_male`: Strong negative correlation with survival.
2. `Pclass`: Moderate negative correlation with survival.
3. `Fare`: Moderate positive correlation with survival.
4. `Cabin`: Moderate positive correlation with survival.
5. `Ticket`: Selected by SelectKBest but has a weaker correlation with survival.

### Conclusion
The analysis suggests that the `Survived` column is the most suitable target variable for prediction. The most relevant features for predicting survival are `Sex_male`, `Pclass`, `Fare`, `Cabin`, and `Ticket`. These features should be considered when building a machine learning model to predict passenger survival.

This report provides a clear understanding of the relationships between different features and highlights the most important ones for predicting the target variable.



"""

In [48]:
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer


# Create a PDF document
pdf_filename = "Feature_Selection_and_Correlation_Analysis_Report.pdf"
doc = SimpleDocTemplate(pdf_filename, pagesize=letter)

# Get a sample stylesheet and define styles
styles = getSampleStyleSheet()
normal_style = styles['Normal']
heading_style = ParagraphStyle(name='Heading', parent=styles['Heading1'], fontSize=14, leading=16, spaceAfter=12, textColor=colors.darkblue)

# Split the content into lines and create Paragraph elements
content = []

# Split lines based on the new lines in the agent_output
lines = agent_output.split('\n')

# Add paragraphs to the content list
for line in lines:
    if line.startswith("###"):
        content.append(Paragraph(line.strip("# "), heading_style))
    elif line.startswith("####"):
        content.append(Paragraph(line.strip("# "), styles['Heading2']))
    elif line.startswith("- "):
        content.append(Paragraph(line, styles['Bullet']))
    elif line.strip():
        content.append(Paragraph(line, normal_style))
    
    # Add a spacer between paragraphs
    content.append(Spacer(1, 12))

# Build the PDF
doc.build(content)

print(f"PDF created successfully with the content of agent_output and saved as '{pdf_filename}'.")


PDF created successfully with the content of agent_output and saved as 'Feature_Selection_and_Correlation_Analysis_Report.pdf'.


In [51]:
import pdfplumber

# Function to extract text using pdfplumber
def extract_text_with_pdfplumber(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
    return text


In [52]:
pdf_file_path = "Feature_Selection_and_Correlation_Analysis_Report.pdf"
pdf_text = extract_text_with_pdfplumber(pdf_file_path)
print(pdf_text)

Feature Selection and Correlation Analysis Report
Possible Target Columns for Prediction
Based on the dataset, the most likely target column for prediction is `Survived`, which indicates whether
a passenger survived or not.
Selected Features for Predicting `Survived`
Using the SelectKBest method, the top 5 features selected for predicting the `Survived` column are:
1. `Pclass`
2. `Ticket`
3. `Fare`
4. `Cabin`
5. `Sex_male`
Correlation Matrix Analysis
The correlation matrix provides insights into the relationships between different features. Here are
some key observations:
- `Survived` has a strong negative correlation with `Sex_male` (-0.543351), indicating that males were
less likely to survive.
- `Survived` has a moderate positive correlation with `Fare` (0.257307) and `Cabin` (0.175690),
suggesting that passengers who paid higher fares and had cabins were more likely to survive.
- `Pclass` is negatively correlated with `Survived` (-0.338481), indicating that passengers in higher
cla

In [66]:
# Define a generalized prefix prompt to guide the agent
prefix_prompt = """
You are a data scientist tasked with building and evaluating different machine learning models based on a report that describes feature selection and correlation analysis.\n
The report provides insights into potential target columns for prediction and lists the most relevant features to use.\n

### Available Tools and Dataset:\n
- You have access to a Python REPL tool with a Pandas DataFrame that has already been pre-processed. The pre-processing includes:\n
    - Handling of missing values\n
    - Encoding of categorical features using Label Encoding or One-Hot Encoding, depending on the number of unique categories\n
    - Scaling of numerical features for consistent input to machine learning models\n
- The dataset is already loaded and available as a Pandas DataFrame.\n

### Task Instructions:\n
1. **Understand the Dataset**: Examine the pre-processed Pandas DataFrame to understand its structure and the columns present.\n
2. **Identify Target and Features**: Use the content of the report provided to determine the target column for prediction. Identify the most relevant features that have been highlighted in the report for predicting the target column.\n
3. **Model Building**: Using the identified target column and relevant features:\n
    - Train different machine learning models. Consider using a variety of algorithms such as:\n
        - Logistic Regression\n
        - Decision Tree\n
        - Random Forest\n
        - Support Vector Machine (SVM)\n
        - Gradient Boosting Classifier\n
        - Any other model that may be suitable based on the data characteristics\n
4. **Model Evaluation**: Evaluate each model's performance using suitable evaluation metrics such as accuracy, precision, recall, F1-score, and cross-validation. Choose the metrics based on the nature of the target column (e.g., binary classification, multiclass classification, regression).\n
5. **Comparison and Analysis**: Compare the results of the different models based on the evaluation metrics. Identify which model performs best and provide insights into why certain models may perform better than others, considering the features and correlations mentioned in the report.\n
6. **Output Generation**: Create a comprehensive output that includes:\n
    - A summary of each model’s performance\n
    - A comparison table or chart showing the metrics for each model\n
    - A conclusion highlighting the most effective model(s) for predicting the target column based on the evaluation metrics\n
7. **Save the Models**: Once you have trained and evaluated the models:\n
    - Save each model to a file using an appropriate format. Consider using:\n
        - **Pickle** (`.pkl` or `.pickle`):\n
          ```python\n
          import pickle\n
          with open('model_name.pkl', 'wb') as file:\n
              pickle.dump(model, file)\n
          ```\n
        - **Joblib** (`.joblib`):\n
          ```python\n
          from joblib import dump\n
          dump(model, 'model_name.joblib')\n
          ```\n
        - **ONNX** (`.onnx`), if applicable:\n
          ```python\n
          import onnx\n
          onnx.save_model(onnx_model, 'model_name.onnx')\n
          ```\n
        - **HDF5** (`.h5`), for Keras models:\n
          ```python\n
          from tensorflow.keras.models import save_model\n
          save_model(model, 'model_name.h5')\n
          ```\n
    - Ensure the file paths and names are specified clearly and are accessible for future use.\n
    - Verify that the model files have been saved correctly by attempting to load them back.\n
    - save the models in the current directory as pickle files.\n

### Additional Considerations:\n
- Be mindful of overfitting, especially with more complex models. Consider using regularization techniques or parameter tuning if necessary.\n
- Ensure the models are interpretable and provide reasoning for feature importance or selection criteria, if applicable.\n
- Document the entire process, including the steps taken for data preparation, model training, evaluation, and conclusions.\n

Use this structure to systematically approach building, evaluating, saving, and comparing machine learning models based on the report provided. The agent should adapt the methodology as necessary depending on the specific features, correlations, and target columns mentioned in the report.\n

"""

In [67]:
df_input = pd.read_csv('transformed_df.csv')

In [68]:
agent_pandas_2 = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-4o", openai_api_key = OPENAI_API_KEY),
    df_input,
    prefix=prefix_prompt,
    verbose=True,
    allow_dangerous_code=True,
    agent_type="openai-tools",
    handle_parsing_errors=True
)

/Users/alymohamed/Desktop/langchain/.venv/lib/python3.11/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [71]:
agent_pandas_2.run(pdf_text)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "from sklearn.model_selection import train_test_split\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier\nfrom sklearn.svm import SVC\nfrom sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score\nimport pandas as pd\n\n# Define the target and features\nfeatures = ['Pclass', 'Ticket', 'Fare', 'Cabin', 'Sex_male']\ntarget = 'Survived'\n\nX = df[features]\ny = df[target]\n\n# Split the data into training and testing sets\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Initialize models\nmodels = {\n    'Logistic Regression': LogisticRegression(),\n    'Decision Tree': DecisionTreeClassifier(),\n    'Random Forest': RandomForestClassifier(),\n    'Gradient Boosting': GradientBoostingClassifier(),\n    'SVM

/Users/alymohamed/Desktop/langchain/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                 Model  Accuracy  Precision    Recall  F1 Score
0  Logistic Regression  0.765363   0.722222  0.702703  0.712329
1        Decision Tree  0.826816   0.802817  0.770270  0.786207
2        Random Forest  0.832402   0.833333  0.743243  0.785714
3    Gradient Boosting  0.815642   0.815385  0.716216  0.762590
4                  SVM  0.692737   0.787879  0.351351  0.485981
Invoking: `python_repl_ast` with `{'query': 'import pickle\nfrom joblib import dump\n\n# Save models\nfor name, model in models.items():\n    # Save with pickle\n    with open(f\'{name.replace(" ", "_").lower()}.pkl\', \'wb\') as file:\n        pickle.dump(model, file)\n    # Save with joblib\n    dump(model, f\'{name.replace(" ", "_").lower()}.joblib\')\n\n# Verify saved models\nloaded_models = {}\nfor name in models.keys():\n    with open(f\'{name.replace(" ", "_").lower()}.pkl\', \'rb\') as file:\n        loaded_models[name] = pickle.load(file)\nloaded_models'}`


{'Logistic Regression': LogisticRegression

'### Model Performance Summary\n\nHere is the performance summary of each model based on the evaluation metrics:\n\n| Model                | Accuracy | Precision | Recall  | F1 Score |\n|----------------------|----------|-----------|---------|----------|\n| Logistic Regression  | 0.765    | 0.722     | 0.703   | 0.712    |\n| Decision Tree        | 0.827    | 0.803     | 0.770   | 0.786    |\n| Random Forest        | 0.832    | 0.833     | 0.743   | 0.786    |\n| Gradient Boosting    | 0.816    | 0.815     | 0.716   | 0.763    |\n| SVM                  | 0.693    | 0.788     | 0.351   | 0.486    |\n\n### Comparison and Analysis\n\n- **Random Forest**: This model achieved the highest accuracy (0.832) and a high F1 Score (0.786), indicating a good balance between precision and recall.\n- **Decision Tree**: This model also performed well with an accuracy of 0.827 and an F1 Score of 0.786.\n- **Gradient Boosting**: This model had a slightly lower accuracy (0.816) but still maintained a goo

In [72]:
# Next Steps: 
# - Create a program with all the agents
# - Fix the agent that does hyperparameter tuning.
# - Test the program with the agents.